# Introduction

In this project, an [OpenStreetMap](https://www.openstreetmap.org) XML dataset of the Raleigh, North Carolina area is extracted from [Map Zen](https://mapzen.com/data/metro-extracts/#raleigh-north-carolina). 

First, I use data munging techniques, such as assessing the quality of the data for validity, accuracy, completeness, consistency and uniformity, to clean the OpenStreetMap data of the Raleigh, North Carolina area. Then the cleaned data is saved as JASON file, and imported into MongoDB for further analysis.

# Data Process

The OpenStreeMap(OSM) XML dataset of Raleigh is about 500 MB. To simplify the data clean process, we first take a systematic sample of elements from original region, and make sure the data process procedure work for the subsampled dataset. 

## Problems in the Data

Through wrangling the sample dataset, I found some two problems:
* Inconsistent street names (e.g., 'St.', 'St', and 'Street')
* Different formats of postcodes (e.g., '27606-3188' and '27606')

For the inconsistent stree names, I changed the abbreviations into full names (e.g. 'St.' or 'St' to 'Street').

For the postcodes, the 5-digits format is applied to all records(e.g. '27606-3188' to '27606').

The two processes are implemented programmatically in the code.

After transforming the XML dataset into JSON format, we have the file with size about 518.5 MB.

# Data Analysis

## Tags Numbers

To analyze the dataset, we first count the number of tags in the XML file, and get the following records:

```
{'bounds': 1,
 'member': 8078,
 'nd': 2498300,
 'node': 2216984,
 'osm': 1,
 'relation': 814,
 'tag': 835000,
 'way': 226555}
```

Then we check if there are any potential problems for each `<tag>`. Generally, all the items fall into four categories: 

```python
'''
  "lower": for tags that contain only lowercase letters and are valid;
  "lower_colon": for otherwise valid tags with a colon in their names;
  "problemchars": for tags with problematic characters;
  "other": for other tags that do not fall into the other three categories.
'''
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
```

We get the number of each category as follow:
```
{'lower': 517443, 'lower_colon': 273752, 'other': 43805, 'problemchars': 0}
```

## Import into MongoDB

After getting the JSON file, we import it into the MongoDB and do some basic statistics about the dataset.

**Import into database:**

* Start a running `mongod` instance at shell: 
> `$mongod`
* Import data into the database at shell:
> `mongoimport --db OpenStreetMap --collection Raleigh_NC --drop --file raleigh_north-carolina.osm.json`

**Count the number of nodes and ways:**

```pyton
print 'Total number of documents: ', collection.find().count()
print 'Number of node:', collection.find({'type': 'node'}).count()
print 'Number of way:', collection.find({'type': 'way'}).count()
```
> Total number of documents:  2443539

> Number of node: 2216984

> Number of way: 226555

This is consistent with the XML dataset records.

**Number one contributor**

```python
a = collection.aggregate([{"$group": {"_id": "$created.user", "count":{"$sum":1}}}, {"$sort":{"count":-1}}, {"$limit":10}])
pprint.pprint(list(a))

[{u'_id': u'jumbanho', u'count': 1581327},
 {u'_id': u'bdiscoe', u'count': 127974},
 {u'_id': u'woodpeck_fixbot', u'count': 117842},
 {u'_id': u'bigal945', u'count': 103333},
 {u'_id': u'JMDeMai', u'count': 92466},
 {u'_id': u'yotann', u'count': 66844},
 {u'_id': u'runbananas', u'count': 42069},
 {u'_id': u'sandhill', u'count': 32089},
 {u'_id': u'MikeInRaleigh', u'count': 22836},
 {u'_id': u'FIM', u'count': 20200}]
```

**Number one contributor with UID**

```python
a = collection.aggregate([{"$group": {"_id": "$created.uid", "count":{"$sum":1}}}, {"$sort":{"count":-1}}, {"$limit":10}])
pprint.pprint(list(a))

[{u'_id': u'38487', u'count': 1581327},
 {u'_id': u'402624', u'count': 127974},
 {u'_id': u'147510', u'count': 117842},
 {u'_id': u'2294834', u'count': 103333},
 {u'_id': u'922360', u'count': 92466},
 {u'_id': u'522859', u'count': 66844},
 {u'_id': u'199905', u'count': 42069},
 {u'_id': u'1694880', u'count': 32089},
 {u'_id': u'2254200', u'count': 22836},
 {u'_id': u'398883', u'count': 20200}]
```

**Number of users appearing only once (having 1 post)**

```python
b = collection.aggregate([{"$group":{"_id":"$created.user", "count":{"$sum":1}}}, {"$group":{"_id":"$count", "num_users":{"$sum":1}}}, {"$sort":{"_id":1}}, {"$limit":1}])
print list(b)

[{u'num_users': 139, u'_id': 1}]

```
**No.1 appearing place names**

```python
a = collection.aggregate([{"$match":{"name":{"$exists":1}}}, {
            "$group":{"_id":"$name","count":{"$sum":1}}
        }, {"$sort":{"count":-1}}, {"$limit":1}])
pprint.pprint(list(a))

[{u'_id': {u'en': u'Raleigh-Durham International Airport'}, u'count': 1}]
```

# Other Ideas About the Dataset

Because there are so many format problems in the dataset, I would suggest the OSM to add some constraints for the user input content. For example, when enter the address, the address name should not contain any abbreviation. If there are some abbreviations, such as 'St', the website should point out the problem or even correct it automatically and ask the user to confirm it.

In order to avoid incorrect or inconsistent data records, the website can implement some search-compare functions. When a user enter a record, such as address, the website can compare the entered address with the existing ones and check if they are consistent or not.

Potential problems for the implementation is that the website may need a very powerful server and fast data search and comparision program to make sure the result can be reached very fast. Otherwise, either the user may need wait for a long time to enter a record, or the automatic correctness procedure cannot be implemented effectively.


# Conclusion

The OSM data of Raleigh, NC area is extracted and cleaned. The XML file is transformed into JSON file, and then imported into MongoDB. A series of queries and analysis is conducted for the JSON file in the MongoDB.

# Appendices

In [5]:
# 1) Subsample the original dataset

import xml.etree.cElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "raleigh_north-carolina.osm"  # Replace this with your osm file
SAMPLE_FILE = "raleigh_north-carolina_sample.osm"

k = 80 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

In [14]:
# 2) Wrangle the data and transform the shape of the data model
import re
import json
import pprint
import xml.etree.cElementTree as ET 

#dataset = 'raleigh_north-carolina_sample.osm'
dataset = 'raleigh_north-carolina.osm'


# For different types of tags
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]

# For address and other tag types
re_addr = re.compile(r'^addr:[^:]*$')
re_xxx = re.compile(r'^[^:]*:[^:]*$')

# Fix unexpected street type
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", 
            "Square", "Lane", "Road","Trail", "Parkway", "Commons"]

# UPDATE THIS VARIABLE
mapping = { "St": "Street",
            "St.": "Street",
            "Rd": "Road",
           "Rd.": "Road",
           "Ct": "Court",
           "Ct.": "Court"
            }


def update_name(name, mapping):
    '''Update name style in mapping dictionary.'''
    tmp = name.split()
    if tmp[-1] in mapping:
        tmp[-1] = mapping[tmp[-1]]
    name = " ".join(tmp)

    return name

def shape_element(element):
    '''Shape the element in XML into data model format.'''
    node = {}
    if element.tag == "node" or element.tag == "way" :
        node['id'] = element.attrib['id']
        node['type'] = element.tag
        if 'visible' in element.attrib:
            node['visible'] = element.attrib['visible']
        # For items in CREATED
        node['created'] = {}
        for x in CREATED:
            if x in element.attrib:
                node['created'][x] = element.attrib[x]
        if 'lon' in element.attrib:
            node['pos'] = [float(element.attrib['lat']), float(element.attrib['lon'])]

        for tag in element.iter('tag'):
        # If not in problemchars
        #if 'k' in element.attrib:
          if not problemchars.match(tag.attrib['k']):
            # If in addr:xxx form

            if re_addr.match(tag.attrib['k']):

                if 'address' not in node:
                    node['address'] = {}
                # print tag.attrib['k']
                if tag.attrib['k'][5:] == 'street':
                    node['address'][tag.attrib['k'][5:]] = update_name(tag.attrib['v'],
                                                                      mapping)
                elif tag.attrib['k'][5:] == 'postcode':
                    node['address'][tag.attrib['k'][5:]] = tag.attrib['v'][0:5]
                else:
                    node['address'][tag.attrib['k'][5:]] = tag.attrib['v']
            
            # If not in xxx:xxx form
            elif re_xxx.match(tag.attrib['k']):
                tmp = tag.attrib['k'].index(':')
                if tag.attrib['k'][:tmp] not in node:
                    node[tag.attrib['k'][:tmp]] = {}
                node[tag.attrib['k'][:tmp]][tag.attrib['k'][tmp+1:]] = tag.attrib['v']

        tmp = []
        for tag in element.iter('nd'):
            if 'ref' in tag.attrib:
                tmp.append(tag.attrib['ref'])
        if len(tmp) > 0:
            node['node_refs'] = tmp

        return node
    else:
        return None


def process_map(file_in, pretty = False):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    data = []
    with open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data


if __name__ == "__main__":
    data = process_map(dataset)
    pprint.pprint(data[0:10])
    
    # Print sample data
    #for x in data:
    #    if 'address' in x:
    #        pprint.pprint(x['address'])
    

[{'created': {'changeset': '7525913',
              'timestamp': '2011-03-11T18:03:13Z',
              'uid': '38487',
              'user': 'jumbanho',
              'version': '4'},
  'id': '21592692',
  'pos': [35.9746754, -78.9086172],
  'type': 'node'},
 {'created': {'changeset': '6118030',
              'timestamp': '2010-10-20T17:35:46Z',
              'uid': '38487',
              'user': 'jumbanho',
              'version': '2'},
  'id': '21595910',
  'pos': [35.9323309, -78.9249163],
  'type': 'node'},
 {'created': {'changeset': '9059469',
              'timestamp': '2011-08-18T19:00:34Z',
              'uid': '398883',
              'user': 'FIM',
              'version': '3'},
  'id': '21595911',
  'pos': [35.9293805, -78.9272018],
  'type': 'node'},
 {'created': {'changeset': '9059469',
              'timestamp': '2011-08-18T19:00:34Z',
              'uid': '398883',
              'user': 'FIM',
              'version': '3'},
  'id': '21595918',
  'pos': [35.9291511, -78.9

In [1]:
# 3) Count tags in dataset
import xml.etree.cElementTree as ET
import pprint

dataset = 'raleigh_north-carolina.osm'

def count_tags(filename):
        '''Iterative parsing to find tags and corresponding numbers.'''
        tree = ET.parse(filename)
        root = tree.getroot()
        dict_tags = {}
        return recursive_count(root, dict_tags)


def recursive_count(root, dict_tag):
    '''Recursively count the number of tags in a root.'''

    if root.tag in dict_tag:
        dict_tag[root.tag] += 1
    else:
        dict_tag[root.tag] = 1

    if len(root) != 0:
        for child in root:
            recursive_count(child, dict_tag)
    return dict_tag

tags = count_tags(dataset)
pprint.pprint(tags)

{'bounds': 1,
 'member': 8078,
 'nd': 2498300,
 'node': 2216984,
 'osm': 1,
 'relation': 814,
 'tag': 835000,
 'way': 226555}


In [5]:
# 4) Check if there are any potential problems for each "<tag>"
'''
  "lower", for tags that contain only lowercase letters and are valid,
  "lower_colon", for otherwise valid tags with a colon in their names,
  "problemchars", for tags with problematic characters, and
  "other", for other tags that do not fall into the other three categories.
'''
import re

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

#dataset = 'raleigh_north-carolina_sample.osm'
dataset = 'raleigh_north-carolina.osm'

def key_type(element, keys):
    if element.tag == "tag":
        if lower.match(element.attrib['k']):
            keys['lower'] += 1
        elif lower_colon.match(element.attrib['k']):
            keys['lower_colon'] += 1
        elif problemchars.match(element.attrib['k']):
            keys['problemchars'] += 1
        else:
            # print element.attrib
            keys['other'] += 1

    return keys


def check_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

keys = check_map(dataset)
pprint.pprint(keys)

{'lower': 517443, 'lower_colon': 273752, 'other': 43805, 'problemchars': 0}


In [1]:
# 5) Using pymongo to connect MongoDB
# Before this step, we need to
# Start a running MongoD instance at shell: `mongod`
# Import the JSON file into MongoDB at shell: 
# `mongoimport --db OpenStreetMap --collection Raleigh_NC --drop --file raleigh_north-carolina.osm.json `

from pymongo import MongoClient
import pprint

client = MongoClient('localhost', 27017)
db = client['OpenStreetMap']
collection = db['Raleigh_NC']

In [21]:
# 6) Count different categories
print 'Total number of documents: ', collection.find().count()
print 'Number of node:', collection.find({'type': 'node'}).count()
print 'Number of way:', collection.find({'type': 'way'}).count()

 Total number of documents:  2443539
Number of node: 2216984
Number of way: 226555


In [5]:
# 8) Number one contributor
a = collection.aggregate([{"$group": {"_id": "$created.user", "count":{"$sum":1}}}, {"$sort":{"count":-1}}, {"$limit":10}])
pprint.pprint(list(a))

[{u'_id': u'jumbanho', u'count': 1581327},
 {u'_id': u'bdiscoe', u'count': 127974},
 {u'_id': u'woodpeck_fixbot', u'count': 117842},
 {u'_id': u'bigal945', u'count': 103333},
 {u'_id': u'JMDeMai', u'count': 92466},
 {u'_id': u'yotann', u'count': 66844},
 {u'_id': u'runbananas', u'count': 42069},
 {u'_id': u'sandhill', u'count': 32089},
 {u'_id': u'MikeInRaleigh', u'count': 22836},
 {u'_id': u'FIM', u'count': 20200}]


In [6]:
# 9) Number one contributor with UID
a = collection.aggregate([{"$group": {"_id": "$created.uid", "count":{"$sum":1}}}, {"$sort":{"count":-1}}, {"$limit":10}])
pprint.pprint(list(a))

[{u'_id': u'38487', u'count': 1581327},
 {u'_id': u'402624', u'count': 127974},
 {u'_id': u'147510', u'count': 117842},
 {u'_id': u'2294834', u'count': 103333},
 {u'_id': u'922360', u'count': 92466},
 {u'_id': u'522859', u'count': 66844},
 {u'_id': u'199905', u'count': 42069},
 {u'_id': u'1694880', u'count': 32089},
 {u'_id': u'2254200', u'count': 22836},
 {u'_id': u'398883', u'count': 20200}]


In [47]:
# 10) Number of users appearing only once (having 1 post)
b = collection.aggregate([{"$group":{"_id":"$created.user", "count":{"$sum":1}}}, {"$group":{"_id":"$count", "num_users":{"$sum":1}}}, {"$sort":{"_id":1}}, {"$limit":1}])
print list(b)

[{u'num_users': 139, u'_id': 1}]


In [8]:
# 11) No.1 appearing place names
a = collection.aggregate([{"$match":{"name":{"$exists":1}}}, {
            "$group":{"_id":"$name","count":{"$sum":1}}
        }, {"$sort":{"count":-1}}, {"$limit":1}])
pprint.pprint(list(a))

[{u'_id': {u'en': u'Raleigh-Durham International Airport'}, u'count': 1}]
